In [ ]:
import ultralytics
from ultralytics import YOLO

In [ ]:
ultralytics.checks()

In [ ]:
# light pre-trained model
model = YOLO('yolov8n.pt')

In [ ]:
# train and validate on VINS dataset
model.train(data='vins.yaml', epochs=50)

In [ ]:
# pick the best result
model = YOLO('runs\\detect\\train\\weights\\best.pt')

In [ ]:
# validate out-of-sample
model.val(data='vins_test.yaml')

In [ ]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14, 10)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
import numpy as np
import os

In [ ]:
img = cv2.imread('runs\\detect\\val\\confusion_matrix.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(np.array(img))
plt.axis('off')
plt.show()

In [ ]:
class_name_to_id_mapping = {
    'Text': 0, 'Icon': 1, 'Image': 2, 'TextButton': 3, 
    'UpperTaskBar': 4, 'EditText': 5, 'PageIndicator': 6, 'CheckedTextView': 7
}
labels = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))
colors = [(0, 0, 255), (0, 255, 0), (0, 255, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255)]


def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)): 
    lw = max(round(sum(image.shape) / 2 * 0.003), 2)
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
    if label:
        tf = max(lw - 1, 1)  # font thickness
        w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image, label, 
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0, lw / 3, txt_color,
            thickness=tf,
            lineType=cv2.LINE_AA
        )
    
def plot_bboxes(image, boxes, labels=[], colors=[]):

    for box in boxes:
        n = int(box[-1])
        box_label(image, box, [], colors[n])

    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
# random prediction example
base_dir = 'datasets\\vins\\test\\images'
images = os.listdir(base_dir)
fn = images[np.random.randint(len(images))]

img = cv2.imread(os.path.join(base_dir, fn))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

res = model.predict(img)[0]
plot_bboxes(img, res.boxes.boxes, labels, colors)

In [ ]:
# random prediction example
img = cv2.imread('../datasets/FinUI/images/086.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

res = model.predict(img)[0]
plot_bboxes(img, res.boxes.boxes, labels, colors)